In [ ]:
from esper.prelude import *
from esper.report import *
from esper.validation import *
import statsmodels.api as sm

NETWORK_COLORS = [
    ('CNN', 'tab:purple'),
    ('FOXNEWS', 'tab:orange'),
    ('MSNBC', 'tab:green')
]

# Introduction
In the news, everyone's got an opinion, but conversely, everyone also has an opinion about the news. This channel is biased, that show only talks about negative news, the whole media covers too much Trump. Opinions are easy to make based on the individuals shows and clips we watch or see shared on social media every day. Now more than ever, it's critical that our understanding of how we produce and consume news media is informed by data, not anecdotes.

At the same time, advances in artificial intelligence and big data systems are now enabling us to analyze complex visual mediums like video at scale. Computer vision and machine learning technologies can automatically watch video to find and recognize individuals, identify visual attributes like clothing, lighting, and setting, as well as break video into segments, e.g. commercials vs. news. Cloud infrastructure combined with big data systems can enable individuals to scale these technologies to automatically annotate thousands or millions of hours of video.

In light of these two trends, we  conducted a study to analyze 70,000 hours of US TV news video over three years provided by the Internet Archive. We sought to identify trends in screen time, speaking time, topic coverage, and more. Our dataset consisted of every video broadcast by CNN, FOX, and MSNBC from January 1, 2015 to December 31, 2017 and included the video, audio, and closed captions. Methodology for each analysis is provided at the end of this report.

# Gender balance
Most of the news is anchors and guests staring into the camera, discussing the topics of the day with one another (or in monologue). We would like to know who is being provided the platform to share their views, along with when/where that occurs, starting with analyzing men vs. women. We looked at two metrics: screen time, or the length of time a person is on screen, and speaking time, or how long a person is talking. For example, if two men and one woman are on screen for 5 seconds with the woman speaking, this would be 10 seconds of male screen time, 5 seconds of female screen time, and 5 seconds of female speaking time. We then compare the proportion of male time to the total time anyone was on screen or speaking.

## Men are seen and heard more than women

In [ ]:
screen_df = pd.DataFrame.from_csv('/app/data/screen_all.csv')
screen_nh_df = pd.DataFrame.from_csv('/app/data/screen_all_nh.csv')
speak_df = pd.DataFrame.from_csv('/app/data/speaking_all.csv')

fig = plt.figure(figsize=(8,8))
plt.suptitle('Gender balance across entire dataset')
plt.axis('off')
def pieplt(ax, df, title):
    pct = df['M%'][0]
    x = [pct, 100-pct]
    wedges, _ = ax.pie(x, labels=['{}%'.format(i) for i in x], colors=['tab:blue', 'tab:red'], explode=[0.02, 0])
    ax.set_title(title)
    return wedges
    
wedges = pieplt(fig.add_subplot(221), screen_df, 'Screen time')    
pieplt(fig.add_subplot(222), screen_nh_df, 'Screen time (no host)')    
pieplt(fig.add_subplot(223), speak_df, 'Speaking time')    
_ = plt.figlegend(wedges, ['Men', 'Women'])

We found that **across the whole dataset, there were about twice as many men as women shown on screen**, and a similar but slightly more equal balance for speaking time (shown above). This is roughly consistent with a previous finding of the same bias in [feature films](https://www.google.com/about/main/gender-equality-films/). There were no significant differences between the channels (FOX/CNN/MSNBC), however we did find substantial variance in individual shows.

## Individual shows substantially differ

In [ ]:
screen_show = pd.DataFrame.from_csv('/app/data/screen_show.csv')
screen_show_nh = pd.DataFrame.from_csv('/app/data/screen_show_nh.csv')
screen_show_jh = pd.DataFrame.from_csv('/app/data/screen_show_jh.csv')
speaking_show = pd.DataFrame.from_csv('/app/data/speaking_show.csv')
speaking_show_nh = pd.DataFrame.from_csv('/app/data/speaking_show_nh.csv')
[screen_show, screen_show_nh, screen_show_jh, speaking_show, speaking_show_nh] = align(
    'show_id', [
        screen_show, 
        screen_show_nh,
        screen_show_jh,
        speaking_show,
        speaking_show_nh
    ]
)
screen_speak_scatter(
    screen_show, 
    screen_show_nh, 
    speaking_show, 
    speaking_show_nh, 
    'show_id', 
    'Gender balance of screen time by show', 
    ['screen']
)

In [ ]:
shows = screen_show.sort_values(by=['M%']).show_id

def get_face(show):
    host_face = Face.objects.filter(is_host=True, person__frame__video__show__name=show) \
        .select_related('person__frame__video')[0]
    return resize(crop(load_frame(host_face.person.frame.video, host_face.person.frame.number, []), host_face), None, 50)

faces = par_for(get_face, shows, progress=False)

#plt.imshow(cv2.cvtColor(np.vstack((np.hstack(faces[:len(faces)/2]), np.hstack(faces[len(faces)/2:]))), cv2.COLOR_RGB2BGR))
plt.imshow(cv2.cvtColor(np.hstack(faces), cv2.COLOR_RGB2BGR))
_ = plt.axis('off')

Above, we plot the percentage of time men are on screen vs. women separated by the top 40 shows. A blue square is the percentage of screen time for men, and a red square is percentage of screen time for women (so blue square + red square = 100%). The line indicates the size and direction of the disparity--a big line indicates a large difference between men vs. women, and blue indicates more men than women (and red vice versa). The shows are sorted by their percentage of men, from least to most. From this graph, we can observe:
* As expected, **female-hosted shows have more female screen/speaking time**. Shows like The Kelly File and The Rachel Maddow Show have more female screen time, mixed-host shows like The Five and FOX & Friends are towards the middle, and male-hosted shows have more men.
* **Of the top 30 shows, only 3 have a majority female screen time** (Outnumbered, Judge Jeanine, Brooke Baldwin), even though 20 shows have at lesat one female host.
* **Several male hosted shows have men outnumbering women at least 3 to 1** (e.g. Tucker Carlson, Chris Hayes).

The next logical question is: how much is the host contributing to these metrics? We would expect the host to be on screen much of the time, but what about the guests? We re-ran the same analysis, but controlled for this effect by eliminating all instances of the host.


## Guests have the same gender balance regardless of show

In [ ]:
screen_speak_scatter(
    screen_show, 
    screen_show_nh, 
    speaking_show, 
    speaking_show_nh, 
    'show_id', 
    'Gender balance of screen time by show, host vs. non-host', 
    ['screen', 'screen_nh']
)
screen_nh_model = sm.OLS(
    screen_show_nh['M%'].tolist(), 
    sm.add_constant(screen_show['M%'].tolist())
).fit()

Here, the 'x' is used to represent screen time without the host, while the square is screen time with the host. Comparing the two data sets, we can see that the screen time _without_ the host is far most consistent from show-to-show, roughly following a horizontal line around 70% male, suggesting that **the gender distribution of non-hosts (guests, extras, etc.) is mostly the same across every show.** 

In fact, there is a statistically significant but small relationship between the gender distribution including the host and without the host (p = {{'{:.5f}'.format(screen_nh_model.pvalues[1])}}). For every 10 percent increase in the ratio of men to women including the host, there is only a {{int(screen_nh_model.params[1]*10)}} percent increase in the ratio without the host. 

We can check whether these trends still hold when looking at speaking time instead of just screen time.

## Speaking time tracks screen time, with exceptions

In [ ]:
screen_speak_scatter(
    screen_show, 
    screen_show_nh, 
    speaking_show, 
    speaking_show_nh, 
    'show_id', 
    'Gender balance by show, screen time vs. speaking time', 
    ['screen', 'speak']
)
model = sm.OLS(
    speaking_show['M%'].tolist(), 
    sm.add_constant(screen_show['M%'].tolist())
).fit()

Here, we represent the speaking time with a triangle, so a blue triangle is the male percentage of speaking time, and a red triangle is the female percentage of speaking time, whereas the squares still represent screen time. From this, we observe:
* **Speaking time mostly tracks screen time.** That is, as the percentage of screen time for men increases, so does the percentage of speaking time for men in accord. For each 10 percent increase in the ratio of men to women in screen time, the ratio in speaking time increases {{int(model.params[1]*10)}} percent (p = {{'{:.1e}'.format(model.pvalues[1])}}).
* **A few shows have substantially greater female speaking time than screen time.** For example, The Rachel Maddow Show, Hannity, and Tucker Carlson all have disproprtionately greater difference between screen/speaking time than other shows. Upon further inspection, it appears that Hannity and Tucker Carlson are artifacts of the speaking detection algorithm--Sean Hannity and Tucker Carlson are consistently misclassified as women. Rachel Maddow, by contrast, appears to simply speak a _lot_ on her show, frequently giving monologues that last the entire hour.

Lastly, these numbers tell us the gender distribution for everything that's every aired on these shows, but what are these men and women actually talking about? Do men talk about different things than women? Using the transcripts, we selected a representative topic for each segment of the video (drawn from the [US News topic list](https://www.usnews.com/topics/subjects)), and then looked at who was talking/on screen during each kind of segment.

## Women talk about stereotypically feminine topics

In [ ]:
def time_to_sec(t):
    [h,m,s] = t.split(':')
    return 3600*int(h) + 60*int(m) + int(s)

# df = screen_show.copy()
# df['host_frac'] = [time_to_sec(a) / float(time_to_sec(b)) for a, b in zip(screen_show_jh.base.tolist(), screen_show.base.tolist())]
# df = df.sort_values(by=['host_frac'])
# df['test'] = list(range(len(df['host_frac'])))
# ax = df.plot('test', 'host_frac', kind='scatter', use_index=False)
# ax.set_xticks(range(len(df.show_id)))
# _ = ax.set_xticklabels(df.show_id, rotation=45, horizontalalignment='right')

In [ ]:
screen_topic = pd.DataFrame.from_csv('/app/data/screen_topic.csv')
screen_topic_nh = pd.DataFrame.from_csv('/app/data/screen_topic_nh.csv')
speaking_topic = pd.DataFrame.from_csv('/app/data/speaking_topic.csv')
speaking_topic_nh = pd.DataFrame.from_csv('/app/data/speaking_topic_nh.csv')
[screen_topic, screen_topic_nh, speaking_topic, speaking_topic_nh] = \
    align('topic', [screen_topic, screen_topic_nh, speaking_topic, speaking_topic_nh])

screen_speak_scatter(
    screen_topic, 
    screen_topic_nh, 
    speaking_topic,
    speaking_topic_nh, 
    'topic', 'Gender balance by topic', 
    ['screen']
)

The graph above shows the gender balance of the top 60 topics by screen time. A clear trend emerges: **women speak more about stereotypically feminine topics, and men speak more about stereotypically masculine topics.** More women speak about feminism, family, and education, while more men speak about congress, terrorism, and the economy. Screen time without the host and speaking time (not shown) also display the same trend.

We also analyzed potential gender disparities in screen/speaking time across days of the week and across the time of day, but found no significant trends, i.e. each day of the week and time of day roughly tracked the overall gender distribution. This suggests that although the news is overall biased, women are not comparatively worse off during prime time hours.

# Topic coverage
With the abolition of the Fairness Doctrine in 1987 shortly preceding the rise of cable news, it has been a well documented phenomenon that today's news networks are more polarized than ever before. Consuming media from a single network presents a very particular view of the world, but we'd like to quantify the extent to which different networks cover different people, events, and topics. One perspective on this asks, given the same topic like gun control, what viewpoints do the networks have and how do they promote them? This requires a nuanced understanding of the language used, so we instead look at a simpler question: who is covering particular topics, people, and phrases and for how long?

In [ ]:
topic_channel_dfs = pickle.load(open('/app/data/topic_channel.pkl', 'rb'))
topic_channel = topic_channel_dfs['duration']

def topic_bar(df, label):
    df = df.sort_values(by=['CNN'], ascending=False).reset_index().head(30)
    ax = plt.gca()
    for i, (network, col) in enumerate(NETWORK_COLORS):
        df.plot.bar('index', network, ax=ax, position=2-i, color=col, width=0.2)
    ax.set_xticks([-1] + range(len(df['index'])))
    ax.set_xticklabels([''] + list(df['index']), rotation=45, horizontalalignment='right')
    ax.set_ylabel('Duration of discussion (s)')
    _ = ax.set_title('Coverage of top 30 {} by channel'.format(label))
    
topic_bar(topic_channel, 'topics')

The above graph shows the length of time provided by each network to covering the 30 most popular topics (note that they are not mutually exclusive, e.g. a piece on liberal voters will likely count for politics, liberals, and voters). From this, we observe:
* The most common topics are political, which is expected both due to the inherently political nature of the networks and the coverage of the 2016 presidential election.
* Fox provides notably greater coverage of liberals and religion than the other two networks.
* MSNBC focuses far more on politics in that it has less coverage of children, crime, family, terrorism, etc. but more coverage of voters, elections, conservatives, and primaries.

We can break this further down by show to identify specific individuals/segments that are likely to cover individual topics.

In [ ]:
topic_show = pd.DataFrame.from_csv('/app/data/topic_show.csv')

def topic_heatmap(df, label, topic_threshold=2000000, key_threshold=3000000):
    sums = df.sum(axis=1)
    df = df.drop(sums[sums < topic_threshold].index).transpose()
    sums = df.sum(axis=1)
    df = df.drop(sums[sums < key_threshold].index)
    for c in df:
        df[c] /= df[c].sum()
    df = df.fillna(0).transpose()

    fig, ax = plt.subplots(figsize=(20,30))
    sns.heatmap(df.transpose(), xticklabels=True, yticklabels=True, ax=ax, linewidths=0.1)
    _ = plt.title('Normalized {} frequency heatmap by show'.format(label))
    
topic_heatmap(topic_show, 'topic')

The above heatmap shows normalized topic coverage scores, e.g. if Rachel Maddow covers immigration for 6 minutes and Sean Hannity covers it for 4 minutes, then they would receive a normalized score of 0.6 and 0.4, respectively, in the immigration column. Certain shows which appear more frequently than others are identified by clear colorful lines (e.g. MSNBC Live, Anderson Cooper 360). Bright squares indicates that a particular show covers a particular topic more than other shows for the same topic. For example, we can see:
* Rachel Maddow covers almost exclusively election issues (candidates, elections, polls, primaries).
* Fox & Friends is the only show that tends to cover more general interest topics (children, dogs, family, football, games, sleep, summer).
* CNN Newsroom with Brooke Baldwin covers an unusual amount of death.

We can further specialize these topics by looking at the mention of individual people and phrases.

In [ ]:
person_channel = pd.DataFrame.from_csv('/app/data/person_channel.csv')
topic_bar(person_channel, 'people')

**TODO: remove DT & HC from the graph.**

The above graph is the same as before, except looks at coverage of people instead of topics. We observe:
* Donald Trump was mentioned more than every other individual combined. 
* Trump was provided notably greater coverage on both CNN and MSNBC than Fox. 
* By contrast, Hillary Clinton, Bill Clinton, and Barack Obama received slightly more coverage on Fox.
* Controversial individuals in the Trump Administration like Michael Flynn, Jared Kushner, and Paul Manafort received substantially more coverage from CNN and MSNBC than Fox.

Again breaking down by show:

In [ ]:
person_show = pd.DataFrame.from_csv('/app/data/person_show.csv')
topic_heatmap(person_show, 'person', topic_threshold=300000, key_threshold=300000)

We can observe that:
* Fox & Friends provided little coverage of anyone in particular except for Chuck Schumer and Nancy Pelosi.
* Rachel Maddow provided unusually high coverage of individuals like Chris Christie, Paul Manafort, and Scott Walker.

Lastly, we can look at the usage of individual phrases.

In [ ]:
phrase_channel = pd.DataFrame.from_csv('/app/data/phrase_channel.csv')
topic_bar(phrase_channel, 'phrases')

MSNBC focus substantially on health care (including the Affordable Care Act). Fox covered the Obama Administration, fake news, and tax cuts. CNN has no particular outlying behavior.

In [ ]:
phrase_show = pd.DataFrame.from_csv('/app/data/phrase_show.csv')
topic_heatmap(phrase_show, 'phrase', key_threshold=100000, topic_threshold=100000)

Some notable trends:
* The "Make America Great Again" catchphrase occurs the most frequently on Fox & Friends and Hannity. Similarly, Hannity is the sole major user of "fake news."
* Both Anderson Cooper and Hannity focus on "collusion," however presumably two sides of the same case.
* Morning Joe is the only show to focus on "Wall Street."
* Fox & Friends is the only show to focus on "small business."
* Chris Hayes focuses on mostly serious topics including "health care" and "immigration reform."

Overall, these analyses support the conclusion that there is indeed a disparity of coverage in topics between shows. No individual show or even channel will cover all topics. Bias in topic coverage largely tracks the political leanings of the respective channels and shows. 

# Clothing
On a lighter note, we also analyzed the clothing styles of anchors on each show, e.g. what kind of clothing they wear (shirt, dress, suit), what color they wear (necktie, hair, clothing), how long the hair is, and so on. We can visualize these clothing attributes by averaging together every image of hosts with those attributes:

In [ ]:
def imshow(path):
    img = cv2.imread(path)
    plt.figure(figsize=(img.shape[1]/100,img.shape[0]/100), dpi=300)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    plt.axis('off')
    
imshow('/app/data/anchor_average.jpg')

These images can confirm basic intuition like women are more likely to wear solid/bright color, men mostly wear suits, and the average person in nearly every category is Caucasian. We can dive deeper into individual clothing trends by looking across time.

## Anchor clothing matches typical seasonal fashion trends

In [ ]:
df = pickle.load(open('/app/data/clothing_color_time.pkl', 'rb'))
df2 = pickle.load(open('/app/data/clothing_style_time.pkl', 'rb'))
df = pd.concat([df, df2], axis=1)
for col in df.columns:
    cmin, cmax = (df[col].min(), df[col].max())
    if cmax - cmin > 0:
        df[col] = (df[col] - cmin) / (cmax - cmin)
df['date'] = df.index

def clothing_time_plot(styles):
    fig = plt.figure(figsize=(12, 6))
    plt.suptitle('Clothing style over time', y=1.02)
    for plotidx, (stylename, styleset) in enumerate(styles):
        year_dates = [df.date[48], df.date[96]]
        ax = fig.add_subplot(len(styles), 1, plotidx+1)    
        for style, color in zip(styleset, ['tab:cyan', 'tab:olive']):
            df.plot('date', style, ax=ax, color=color)
        ax.axvline(x=year_dates[0], linestyle='--')
        ax.axvline(x=year_dates[1], linestyle='--')
        ax.set_title(stylename)
        ax.set_xlabel('Date')
        ax.set_ylabel('Normalized frequency')
        ax.legend(loc='upper right')
    plt.tight_layout()
    
clothing_time_plot([('summer', ('white', 't-shirt')), ('winter', ('black', 'sweater'))])

The above graph shows the normalized frequency of occurrence for when all hosts wear certain styles of clothes. Normalized means 1 is when people are wearing a style of clothes most often, and 0 the least--it does not imply how often these styles occur relative to one another. Here, we can see that our **anchors follow established seasonal trends**, i.e. they are more likely to wear lighter colors (white) and clothing (t-shirt) in the summer, and more likely to wear darker colors (black) and heavier clothing (sweaters) in the winter. These are across all news, but if we break by network, we can find distinguishing trends between the channels.

## Each network has their own unique style for female hosts

In [ ]:
def clothing_network_plot(fmt, styles, title):
    fig = plt.figure(figsize=(12, 3*len(styles)))
    plt.suptitle('{} style over time across channels for women'.format(title), y=1.02)
    for plotidx, style in enumerate(styles):
        df = pickle.load(open(fmt.format(style), 'rb')).reset_index()
        year_dates = [df.date[48], df.date[96]]
        ax = fig.add_subplot(len(styles), 1, plotidx+1)    
        for network, col in NETWORK_COLORS:
            df.plot('date', network, ax=ax, color=col)
        ax.set_ylim(0, 1)
        ax.axvline(x=year_dates[0], linestyle='--')
        ax.axvline(x=year_dates[1], linestyle='--')
        ax.set_title(style)
        ax.set_xlabel('Date')
        ax.set_ylabel('Frequency')
        ax.legend(loc='upper right')
    plt.tight_layout()
     
clothing_network_plot('/app/data/clothing_style_{}_channel.pkl', ['suit', 'dress', 'shirt'], 'Clothing')

The above graph shows _unnormalized_ clothing frequency for female hosts, e.g. an 0.8 dress score on the orange line means 80% of Fox female hosts wore dresses in that week. Men largely wear suits all the time, only choosing to vary 
their necktie preference, but women exhibit much greater variance in their clothing choices. Specifically, we can see that **more women on Fox wear dresses than the other channels, whereas CNN prefers shirts and MSNBC prefers suits**. This tracks with previous hypotheses that Fox News women are more likely to wear dresses, but what about the famous [blond](http://www.stevedennie.com/those-foxnews-blondes/) [hair](https://www.thecut.com/2017/08/politics-of-blonde-hair-from-persephone-to-ivanka-trump.html)?

## CNN has as many blonde hosts as Fox

In [ ]:
clothing_network_plot('/app/data/clothing_hair_{}_channel.pkl', ['black', 'blond'], 'Hair')

Surprisingly, **CNN has just as many blonde hosts as Fox**. The only outlier is MSNBC, who has more black haired women by comparison.

# Cinematography

Shots, or contiguous sequences of frames from a single perspective, underpin the cinematography of produced video content, most notably in movies but also in TV news. Shot-based statistical analyses usually look at either the kind of shot (short, long, wide, etc.) or the length of the shot, such as the [Cutting et al.](http://people.psych.cornell.edu/~jec7/pubs/iperception.pdf) study which found that the average length of shots in cinema has decreases over the years to about 3.5 seconds per shot today. In turn, we sought to analyze what shot lengths tell us about the cinematography underlying TV news.

## There many short shots and a few long shots

In [ ]:
shot_meds = pickle.load(open('/app/data/shot_medians.pkl', 'rb'))
shot_hist = pickle.load(open('/app/data/shot_histogram.pkl', 'rb'))
plt.title('Histogram of shot lengths')
plt.semilogx(shot_hist)
plt.ylabel('Number of shots')
_ = plt.xlabel('Shot length (s)')

The graph above shows a histogram of the number of shots at a given length, from 1 second to an hour (x-axis on a log scale). The linear relationship between the log of shot length and number of shots on the graph indicates an exponential relationship in the true values, i.e. that there are exponentially many more short shots (1-10 seconds) than long shots (>10 seconds).

## TV news is similar to movies, commercials are twice as fast

If we divide the TV news videos into news and commercials, we find that the two kinds of video have median shot lengths of {{'{:.3f}'.format(shot_meds['nocom'])}} seconds and {{'{:.3f}'.format(shot_meds['withcom'])}} seconds, respectively. Compared to the 3.5s average length for movies, this suggests that **the pacing of shot lengths in TV news is similar to that used in today's films**. By contrast, commercials are extremely fast paced, having a median shot length more than twice as fast as movies and news.

## Each channel has a distinctly different shot lengths

Dividing the data by channel, we find that Fox has a median shot length of {{'{:.3f}'.format(shot_meds['channels']['FOXNEWS'])}}s, compared with {{'{:.3f}'.format(shot_meds['channels']['CNN'])}}s for CNN and {{'{:.3f}'.format(shot_meds['channels']['MSNBC'])}}s for MSNBC. This suggests that MSNBC has the slowest pacing of the three, and Fox the fastest.

In [ ]:
shot_recall = 0.98
shot_seconds = 3600*6

commercial_precision = 0.99
commercial_recall = 0.99

big_face = face_validation(
    'Faces height > 0.2', lambda qs: qs.annotate(height=F('bbox_y2') - F('bbox_y1')).filter(height__gte=0.2),
    noprint=True)
(face_precision, face_recall, labeled_frames) = big_face[2]

_, big_gender_cmat = gender_validation('Gender w/ face height > 0.2', big_face, noprint=True)

speaking_cmat, total_duration = speaking_validation(noprint=True)

clothing_table = '''wearing jacket & 1.0 
clothing category  & 0.91 
sleeve length & 0.96 
neckline shape & 1.0 
collar presence & 1.0 
wearing scarf & 1.0 
wearing necktie & 0.96 
clothing pattern & 0.92 
major color & 0.81 
wearing hat & 1.0 
wearing glasses & 1.0 
multiple layers  & 0.96 
necktie color & 0.28 
necktie pattern & 0.84  
hair color & 0.53  
hair length & 0.89'''
clothing_table = [s.strip().split(' & ') for s in clothing_table.split('\n')]

def pct(n):
    return int(n*100)

# Methodology

We used machine learning methods to compute many of the underlying features used in the analysis above. As with all ML, these results are not perfect, and we performed validation by manually hand-labeling data to understand the extent of the correctness in methods used.

## Screen time

To estimate the screen time of individuals, we used a three step proces:
1. Segment each video into shots.
2. Detect faces in the middle frame from each shot.
3. Estimate gender of each detected face.

If a face is detected in the representative frame of the shot, we assume that the face is also in every other frame in the shot, i.e. assign that face a number of seconds equal to the shot length. We used a simple custom histogram-based shot detection methods with {{int(shot_recall*100)}}% recall based on {{int(shot_seconds/3600.0)}} hours of hand-labeled video. 

For face detection, we used the MTCNN face detection network and model in the [facenet](https://github.com/davidsandberg/facenet) repository. We handlabeled faces and genders in {{labeled_frames}} randomly sampled frames and found the detector has {{pct(face_precision)}}% precision and {{pct(face_recall)}}% recall for faces with height > 0.2.

For gender detection, we used the [rude-carnie](https://github.com/dpressel/rude-carnie) gender detection network. Our handlabeled dataset produced the following confusion matrix:

In [ ]:
plot_confusion_matrix(
    big_gender_cmat, [d['name'] for d in Gender.objects.values('name').order_by('id')], normalize=True)

Here, the labels are M, F, and U for male, female, and unknown. People of unknown gender were excluded from the gender analysis. The misclassification rates are similar for both men and women, with men being misclassfied slightly more often (15% for men vs. 12% for women).

Additionally, we detected and removed commercials from all analyses above, except where noted otherwise. Our method relied on a number of heuristics such as the existence of black frames before the beginning of a commercial block or the absence of a transcript in the closed captioning. Our commercial detector has {{pct(commercial_precision)}}% precision and {{pct(commercial_recall)}}% recall.

## Speaking time

To estimate the speaking time of individuals, we divided the audio stream into speech/non-speech, segmented the speech into segments containing separate speakers, and then estimated the gender for each speech segment. These steps were all performed end-to-end in a single pass by the [LIUM](http://www-lium.univ-lemans.fr/diarization/doku.php/) speaker diarization tool.

We hand-labeled speech/non-speech, speaker boundaries, and speaker genders in {{int(total_duration/3600.0)}} hours of randomly sampled 3-minute segments of audio, computing the following confusion matrix:

In [ ]:
plot_confusion_matrix(
    speaking_cmat, [d['name'] for d in Gender.objects.values('name').order_by('id')], normalize=True)

The speech labeling tool has near-perfect classification accuracy for women, but a 13% misclassification rate for men, meaning all speaking time estimates are likely to be slightly more female-biased than the true estimate.

## Host detection

To classify a person as the host or non-host, we start by computing the a 128-dimensional feature-space embedding of each face using the [FaceNet](https://github.com/davidsandberg/facenet) network, and cluster the faces within each video using a standard k-means algorithm. Using the hypothesis that a host is uniquely likely to appear frequently throughout the show, we identify face clusters as hosts if faces in the cluster are widely spread in time.

We have not yet validated the host detector with a handlabeled dataset, but visual inspection suggests that our method works most of the time.

## Clothing

To classify the clothing hosts wear into categories, we used a modified version of the [StreetStyle](http://streetstyle.cs.cornell.edu/) clothing classifier trained on the StreetStyle-27k dataset as well as a hand-labeled set of 5000 randomly sampled images of hosts from TV news. We also add necktie color/pattern and hair color/length as attributes (not present in the prior StreetStyle classifier). Classifier accuracies on the TV news host validation set are below.

In [ ]:
df = pd.DataFrame(clothing_table)
df.columns = ['Attribute', 'Accuracy']